<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/Clasificacion_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JFK Delay prediction

Importe librerias

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format


También cargaremos todas las funciones de preprocesamientoo de scikit-learn, contenidas en el **scikit preprocessing**.


In [5]:
from sklearn import preprocessing

Y aprovecharemos para comprobar la **versión instalada de scikit-learn**.

In [2]:
from sklearn import __version__ as sklearn_version
print('La versión instalada de scikit-learn es {}.'.format(sklearn_version))

La versión instalada de scikit-learn es 1.6.1.


Antes de empezar a repasar la Machine Learning Checklist, importatemos el DataSet de trabajo (**JFKFlights**). Este Dataset se encuentra publicado en la página oficial del Departamento de transporte de Estados Unidos , aunque para facilitar el trabajo lo hemos trasladado a un link de descarga en internet (repositorio público en GitHub).

In [3]:
url1 = 'https://bit.ly/3bf6QNP'
url2 = 'https://bit.ly/3kLR2p3'
url_weather = 'https://raw.githubusercontent.com/Sitg35/Binary-Classifier-JFK/refs/heads/main/NYC_Weather_2016_2022.csv'

In [6]:
df1= pd.read_csv(url1, index_col=0)
#lee la url el csv, con el df1= hacemos q se guarde

In [4]:
df2= pd.read_csv(url2, index_col=0)

In [7]:
df3= pd.read_csv(url_weather, index_col=0)

Unir dos csv, se tuvieron q partir por tamaño

In [8]:
df= pd.concat([df1, df2], ignore_index=True)

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
OP_CARRIER_AIRLINE_ID,"237,644.00","20,152.00",297.88,"19,690.00","19,790.00","20,363.00","20,409.00","21,171.00"
CRS_DEP_TIME,"250,152.00","1,334.61",529.10,3.00,915.00,"1,258.00","1,744.00","2,359.00"
TAXI_OUT,"232,641.00",18.19,9.61,1.00,12.00,16.00,21.00,166.00
CRS_ARR_TIME,"250,152.00","1,419.13",580.99,1.00,944.00,"1,455.00","1,914.00","2,400.00"
ARR_DEL15,"250,152.00",0.21,0.41,0.00,0.00,0.00,0.00,1.00
CRS_ELAPSED_TIME,"250,152.00",195.65,97.37,47.00,109.00,167.00,306.00,595.00
DISTANCE,"250,152.00","1,258.84",898.12,94.00,427.00,"1,028.00","2,248.00","4,983.00"


In [10]:
df.reset_index(drop=True, inplace=True)

In [18]:
df_weather=pd.read_csv(url_weather, index_col=0) #granularidad fecha y hora
print(df_weather)

                  temperature_2m (°C)  precipitation (mm)  rain (mm)  \
time                                                                   
2016-01-01T00:00                 7.60                0.00       0.00   
2016-01-01T01:00                 7.50                0.00       0.00   
2016-01-01T02:00                 7.10                0.00       0.00   
2016-01-01T03:00                 6.60                0.00       0.00   
2016-01-01T04:00                 6.30                0.00       0.00   
...                               ...                 ...        ...   
2022-10-25T19:00                  NaN                 NaN        NaN   
2022-10-25T20:00                  NaN                 NaN        NaN   
2022-10-25T21:00                  NaN                 NaN        NaN   
2022-10-25T22:00                  NaN                 NaN        NaN   
2022-10-25T23:00                  NaN                 NaN        NaN   

                  cloudcover (%)  cloudcover_low (%)  cloudcove

In [13]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59760 entries, 2016-01-01T00:00 to 2022-10-25T23:00
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   temperature_2m (°C)    59592 non-null  float64
 1   precipitation (mm)     59592 non-null  float64
 2   rain (mm)              59592 non-null  float64
 3   cloudcover (%)         59592 non-null  float64
 4   cloudcover_low (%)     59592 non-null  float64
 5   cloudcover_mid (%)     59592 non-null  float64
 6   cloudcover_high (%)    59592 non-null  float64
 7   windspeed_10m (km/h)   59592 non-null  float64
 8   winddirection_10m (°)  59587 non-null  float64
dtypes: float64(9)
memory usage: 4.6+ MB


In [22]:
pd.to_datetime(df_weather['time'])

KeyError: 'time'